# Interatomic Potential Repository Content Manager

This is a working Notebook providing a single location for adding and modifying content in the potentials.nist.gov database.

Notebook outline:

0. Setup: Python library imports and defining global parameters.

1. Citations: Load, add and modify citation information associated with potential models. Each potential should have at least one citation, with the first being the "primary" one for generating IDs. For unpublished potentials, use the unpublished citation option.

2. Potentials: Load, add and modify potential information.  A "potential" should be a unique parameterization with an associated citation. Note that saving to the database is done in step #4.

3. Implementations: Load, add and modify the implementations of a potential. An "implementation" is a specific version/representation of a potential in a specific format. Each potential can have zero, one or multiple implementations.

4. Save potentials: Review the new/modified potentials and save the content to the database.

5. Actions: Actions are used to list changes to the repository. If a potential is added/modified, a corresponding action should also be added.

6. Requests: A request is listed on the repository whenever someone asks for a model that we do not have.

## 0. Setup

### Library imports

In [1]:
import uuid
import datetime
import shutil
from pathlib import Path
import socket

import potentials
from potentials.record.PotentialLAMMPS import PotentialLAMMPS

import numpy as np
import pandas as pd

# Jupyter display libraries
from IPython.display import display, HTML

### Global parameters

- __complete_potentials__ is a compiled list of the potentials whose addition/changes were uploaded to the database.  This is used by step #5 for reporting website changes.  This should be reset after creating an Action for meaningful sets of potentials.

- __db__ is the Database object for accessing and uploading records to the database.  remote_name accesses the saved "potentials" settings, which has sign-in info for https://potentials.nist.gov.

- __workspace__ is the workspace to assign to the uploaded records.  If None, then they are only accessible to the submitter until the workspace is changed.  If 'Global Public Workspace', anyone can see the records after uploading.

In [2]:
complete_potentials = []

In [3]:
db = potentials.Database(local=True, remote=True, remote_name='potentials')

In [4]:
workspace = 'Global Public Workspace'

- - -

## 1. Citations

### 1.1 Build or load citation

#### Option #1: For existing citations and/or new dois

In [5]:
doi = '10.1103/PhysRevMaterials.5.113606'

citation = db.fetch_citation(doi, verbose=True)

Matching record retrieved from local


#### Option #2: For new doi-less citations

In [7]:
# This is the general case for a DOI-less citation

citation = potentials.load_record(
    'Citation',

    # Select doctype
    doctype = 'journal',
    #doctype = 'book',
    #doctype = 'report'
    #doctype = 'thesis',
    #doctype = 'conference proceedings',
    #doctype = 'unspecified',

    # Fill in metadata (except for authors)              
    title = 'Angular-dependent interatomic potential for large-scale atomistic simulation of W-Mo-Nb ternary alloys',
    author = 'S. Starikov and P. Grigorev and P. Olsson',
    year = 2024,
    journal = 'Computational Materials Science',
    pages = 'accepted',
    abstract=' '.join(['We present a new classical interatomic potential designed for simulation of',
                       'the W-Mo-Nb system. The angular-dependent format of the potential allows',
                       'for reproduction of many important properties of pure metals and complex',
                       'concentrated alloys with good accuracy. Special attention during the development',
                       'and validation of the potential was paid to the description of vacancies,',
                       'screw dislocations and planar defects, as well as thermo-mechanical',
                       'properties. Here, the applicability of the developed model is demonstrated',
                       'by studying the temperature dependence of the elastic moduli and average',
                       'atomic displacement in pure metals and concentrated alloys up to the melting point.'])
)

# Add author info
citation.add_author(givenname = 'S',  surname = 'Starikov',  suffix = None)
citation.add_author(givenname = 'P',  surname = 'Grigorev',  suffix = None)
citation.add_author(givenname = 'P',  surname = 'Olsson',  suffix = None)

citation.name = citation.year_authors
citation.build_model()
print(citation.model.json(indent=4))

{
    "citation": {
        "document-type": "journal",
        "title": "Angular-dependent interatomic potential for large-scale atomistic simulation of W-Mo-Nb ternary alloys",
        "author": [
            {
                "given-name": "S",
                "surname": "Starikov"
            },
            {
                "given-name": "P",
                "surname": "Grigorev"
            },
            {
                "given-name": "P",
                "surname": "Olsson"
            }
        ],
        "publication-date": {
            "year": 2024
        },
        "abstract": "We present a new classical interatomic potential designed for simulation of the W-Mo-Nb system. The angular-dependent format of the potential allows for reproduction of many important properties of pure metals and complex concentrated alloys with good accuracy. Special attention during the development and validation of the potential was paid to the description of vacancies, screw dislocations and 

In [9]:
# This is the specific case for an unpublished potential (usually from Mendelev)

citation = potentials.load_record(
    'Citation',
    name = '2024--Mendelev-M-I--Ni-Co-Cr',
    doctype = 'unspecified',    
    title = 'to be published',
    year = 2024,
)

citation.add_author(givenname = 'M.I.',  surname = 'Mendelev',  suffix = None)

citation.build_model()

DataModelDict([('citation',
                DataModelDict([('document-type', 'unspecified'),
                               ('title', 'to be published'),
                               ('author',
                                DataModelDict([('given-name', 'M.I.'),
                                               ('surname', 'Mendelev')])),
                               ('publication-date',
                                DataModelDict([('year', 2024)])),
                               ('bibtex',
                                '@unpublished{2024--Mendelev-M-I,\n author = {Mendelev, M.I.},\n title = {to be published},\n year = {2024}\n}\n')]))])

#### Option #3: From local directory

In [ ]:
libdir = Path('E:/website/LAMMPS potentials/2nn/biblib')

citations = []
for bibfile in libdir.glob('*.bib'):
    with open(bibfile, encoding='UTF-8') as f:
        citations.append(potentials.load_record('Citation', model=f.read()))

#### Option #4: From currently loaded potentials

In [ ]:
#citation = complete_potentials[0].citations[0]
#print(citation.year_authors)

### 1.2 Investigate and modify

- Add abstract

- Check and replace garbled latex or other symbols in authors, title and abstract

- Verify correct year and page number(s)

#### 1.2.1 Generate html and list content fields

In [6]:
print(citation.year_authors)
print(citation.name)
print()
citation.html(render=True)
print()
for key, value in citation.bibdict.items():
    print(key+':', value)

2021--Meng-F-S-Du-J-P-Shinzato-S-et-al
10.1103_physrevmaterials.5.113606




ID: 2021--Meng-F-S-Du-J-P-Shinzato-S-et-al
ENTRYTYPE: article
author: Meng, F.-S. and Du, J.-P. and Shinzato, S. and Mori, H. and Yu, P. and Matsubara, K. and Ishikawa, N. and Ogata, S.
title: General-purpose neural network interatomic potential for the 𝛼-iron and hydrogen binary system: Toward atomic-scale understanding of hydrogen embrittlement
journal: Physical Review Materials
year: 2021
volume: 5
number: 11
pages: 113606
month: November
doi: 10.1103/physrevmaterials.5.113606


#### 1.2.2 Modify and update fields

Any valid bibtex fields can be added, but citation rendering relies only on the primary fields. 

In [7]:
citation.title = 'General-purpose neural network interatomic potential for the 𝛼-iron and hydrogen binary system: Toward atomic-scale understanding of hydrogen embrittlement'
citation.abstract = ' '.join([
    "To understand the physics of hydrogen embrittlement at the atomic scale, a general-purpose neural network interatomic potential (NNIP) for the 𝛼-iron and hydrogen binary system is presented. It",
    "is trained using an extensive reference database produced by density functional theory (DFT) calculations. The NNIP can properly describe the interactions of hydrogen with various defects in",
    "𝛼-iron, such as vacancies, surfaces, grain boundaries, and dislocations; in addition to the basic properties of 𝛼-iron itself, the NNIP also handles the defect properties in 𝛼-iron, hydrogen behavior",
    "in 𝛼-iron, and hydrogen-hydrogen interactions in 𝛼-iron and in vacuum, including the hydrogen molecule formation and dissociation at the 𝛼-iron surface. These are superb challenges for the existing",
    "empirical interatomic potentials, like the embedded-atom method based potentials, for the 𝛼-iron and hydrogen binary system. In this study, the NNIP was applied to several key phenomena necessary for",
    "understanding hydrogen embrittlement, such as hydrogen charging and discharging to 𝛼-iron, hydrogen transportation in defective 𝛼-iron, hydrogen trapping and desorption at the defects, and",
    "hydrogen-assisted cracking at the grain boundary. Unlike the existing interatomic potentials, the NNIP simulations quantitatively described the atomistic details of hydrogen behavior in the defective",
    "𝛼-iron system with DFT accuracy."])
citation.pages = '113606'

citation.build_bibtex()
citation.build_model()
print(citation.year_authors)
print(citation.name)
print()
citation.html(render=True)

2021--Meng-F-S-Du-J-P-Shinzato-S-et-al
10.1103_physrevmaterials.5.113606



#### 1.2.3 Review and Save Citation

In [8]:
db.save_citation(citation, overwrite=True, verbose=True)
db.upload_citation(citation, workspace=workspace, overwrite=True, verbose=True)

Citation record named 10.1103_physrevmaterials.5.113606 added to /home/lmh1/library
Citation record named 10.1103_physrevmaterials.5.113606 added to https://potentials.nist.gov
Citation record named 10.1103_physrevmaterials.5.113606 assigned to workspace Global Public Workspace


In [6]:
#db.delete_citation(citation, local=True, remote=True, verbose=True)

Citation record named 10.1063_5.0139010 deleted from https://potentials.nist.gov/


In [7]:
citations = []

In [8]:
citations.append(citation)

- - -

## 2. Potentials

### 2.1 Build or load potential

#### Option #1: Build new potential using citation(s)

- __elements__ (*list*) The element models used by the potential.
- __citations__ (*list*) The citation(s) to associate with the potential.
- __notes__ (*str, optional*) Notes providing more information on the potential, such as usage notes and how the potential relates to other potentials.
- __key__ (*str, optional*) UUID4 key to assign to the potential.  If None, a new key will be generated.
- __othername__ (*str, optional*) Specifies an alternate description of the interactions besides the list of elements.  Examples include if the model is "universal", designed for a specific compound, or is a coarse-grain representation.
- __fictional__ (*bool, optional*) Flag indicating if the model is "fictional", i.e. purposefully fit to unrealistic properties. 
- __modelname__ (*str, optional*) Used if citation + element info is not enough to create a unique ID, e.g. a publication lists multiple parameterizations of the same element. The value should make it clear which version it is associated with, and correspond to how the authors refer to it when possible.
- __recorddate__ (*datetime.date, optional*) Date to assign to the record.  If not given, will use today's date.

In [11]:
potential = potentials.load_record(
    'Potential',
    
    elements = ['Fe', 'H'],
    #fictionalelements = ['Al'],
    #othername = Ge60Se40',
    #modelname='1',  DOESN'T DO ANYTHING AT THE MOMENT!!!!!
    #recorddate=datetime.date(2019, 12, 24)
    
    
    notes = ' '.join([
        'Fan-Shun Meng notes that "This potential was designed for the general purpose usage of the 𝛼-Fe-H binary system. Additionally, the potential also can be used for pure 𝛼-Fe. To',
        'use the potential in LAMMPS, the pair_style of hdnnp should be adopted, and the package of ML-HDNNP should be compiled(see ML-HDNNP documentation)."'
    ]),
)

potential.citations.extend(citations)
potential.name = potential.defaultname
potential.url = f'https://potentials.nist.gov/pid/rest/local/potentials/{potential.name}'

potential.build_model()
None

#### Option #2: Select existing potential

In [5]:
potential = db.get_potential(name='potential.2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H', verbose=True)

Matching record retrieved from local


In [52]:
#db.delete_potential(potential, local=True, remote=True, verbose=True)

Potential record named potential.2021--Arora-G-Bonny-G-Castin-N-Aidhy-D-S--Ni-Fe-Cr-Pd deleted from C:\Users\lmh1\Documents\library
Potential record named potential.2021--Arora-G-Bonny-G-Castin-N-Aidhy-D-S--Ni-Fe-Cr-Pd deleted from https://potentials.nist.gov/


#### Option #3: Get currently loaded potential

In [ ]:
#potential = complete_potentials[1]
#print(potential.name)

### 2.2 Investigate and modify

#### 2.2.1 Generate html and list content fields

In [6]:
potential.html(render=True)

#### 2.2.2 Modify and update fields

Same fields as above

In [60]:
#potential.key = 
#potential.recorddate = datetime.date()
#potential.elements = []
#potential.othername = 'water'
#potential.fictional = 
#potential.modelname = 
potential.notes = ' '.join(['Gaurav Arora notes that "This is one of the first types of potentials used to study radiation defects in alloys containing Pd and is a modified version of',
                            '2018--Bonny-G-Chakraborty-D-Pandey-S-et-al--Ni-Fe-Cr-Pd. This potential was specifically developed to study defect energetics, such as vacancy formation energies, binding energies, voids, stacking fault tetrahedra (SFTs) formation, and other radiation defects in high entropy alloys.'])
#potential.notes += ' The associated publication has a supplementary information document which includes thorough testing of library appropriateness of liquid properties, and comments and analyses on numerical stability and convergence'
potential.notes += '"'
#potential.citations[0] = citation
#potential.name = f'potential.{potential.id}'
potential.build_model()
None

## 3. Implementations

### 3.1 List all current implementation types

List is given to help keep type values (relatively) uniform. Only needs to be called after new types have been added.

In [19]:
pots = db.get_potentials(remote=False)
imptypes = []
for pot in pots:
    for imp in pot.implementations:
        imptypes.append(imp.type)
imptypes = np.unique(imptypes)

print('All existing implementation types:')
for imptypes in imptypes:
    print('   ', imptypes)

All existing implementation types:
    ADP tabulated functions
    ASE calculator
    Dynamo MEAM
    EAM setfl
    EAM tabulated functions
    Equations
    FORTRAN
    Finnis-Sinclair tables
    GAP
    GULP
    IMD option EAM
    LAMMPS pair_style MEAM
    LAMMPS pair_style adp
    LAMMPS pair_style aenet (custom)
    LAMMPS pair_style agni
    LAMMPS pair_style bop
    LAMMPS pair_style comb3
    LAMMPS pair_style eam
    LAMMPS pair_style eam/alloy
    LAMMPS pair_style eam/cd
    LAMMPS pair_style eam/fs
    LAMMPS pair_style eam/he
    LAMMPS pair_style edip
    LAMMPS pair_style edip/multi
    LAMMPS pair_style eim
    LAMMPS pair_style extep
    LAMMPS pair_style hybrid table linear 1000 eam/alloy
    LAMMPS pair_style hybrid/overlay eam/alloy eam/fs
    LAMMPS pair_style hybrid/overlay zbl eam/alloy
    LAMMPS pair_style hybrid/overlay zbl snap
    LAMMPS pair_style lcbop
    LAMMPS pair_style meam
    LAMMPS pair_style meam (modified)
    LAMMPS pair_style meam/spline
    LA

### 3.2 Build or load implementation

#### 3.2.1 List ids for current potential and implementations

In [7]:
print('suggested imp prefix:', potential.impid_prefix)
print('implementation ids:')
for i, implementation in enumerate(potential.implementations):
    print(f'{i}: {implementation.id} {implementation.type}')

suggested imp prefix: 2021--Meng-F-S--Fe-H
implementation ids:
0: 2021--Meng-F-S--Fe-H--LAMMPS--ipr1 LAMMPS pair_style hdnnp


In [86]:
#del potential.implementations[0]

#### 3.2.2 Build new implementation

- __type__ (*str*) Describes the type of the implementation. Use one of the listed values above if possible.
- __id__ (*str*) Unique human-readable id for the implementation. For new content, should be derived from the associated potential id:
    - Remove all authors except for the first
    - Add a simple format descriptor: LAMMPS, GULP, table, parameters, FORTRAN, etc.
    - Add a version descriptor: ipr1, ipr2, etc
- __notes__ (*str, optional*) Notes on the implementation.  This includes where the file(s) came from, who created them, testing info, how it differs from other versions, etc.
- __key__ (*str, optional*) UUID4 key to assign to the implementation.  If None, a new key will be generated.
- __status__ (*str, optional*) Indicates the status of the implementation. Available values are 
    - "active" (default) indicates a current implementation.
    - "superseded" indicates an implementation that is still consistent with the potential model, but has minor issues that were fixed by a newer implementation.
    - "retracted" indicates an implementation that was identified as being an invalid representation of the potential model.
- __date__ (*datetime.date, optional*) The date that the implementation was submitted or added.  If not given, will use today's date.

In [14]:
potential.add_implementation(
    type = 'LAMMPS pair_style hdnnp (custom)',
    id = f'{potential.impid_prefix}--LAMMPS--ipr1',
    notes = ' '.join([
        'These files were provided by Fan-Shun Meng on August 30, 2024.'
    ]),
    #key='0cdc699f-e78a-49db-8606-7a33c98c2184',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
index = len(potential.implementations) - 1

In [27]:
potential.add_implementation(
    type = 'ASE calculator',
    id = f'{potential.impid_prefix}--equations--ipr1',
    notes = ' '.join([
        'This file and equations were provided by Alexey Verkhovtsev on March 19, 2024.  The .pdf collects the equation and parameter values as described in the paper.  The .pot file are the parameters as used by the MBN Explorer software package.'
    ]),
    #key='0cdc699f-e78a-49db-8606-7a33c98c2184',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
index = len(potential.implementations) - 1

In [28]:
kimid = 'MO_959249795837'
implementation = potentials.Implementation(
    type='OpenKIM',
    id=kimid,
    notes=' '.join(['Listing found at https://openkim.org.']),
    #notes=' '.join(['Listing found at https://openkim.org.  This KIM potential is based on a parameter file with identical parameter values as 1989--Tersoff-J--Si-Ge--LAMMPS--ipr1.']),
    #notes=' '.join(['Listing found at https://openkim.org. This KIM potential is implemented from the analytical expressions rather than a tabulated parameter file.'])#  The parameter file that this KIM potential is based on has slightly different values due to precision rounding than 2017--Purja-Pun-G-P--Si--LAMMPS--ipr1.']),
    key='da76eb36-7ca1-421a-bd64-8f6bdd905b15',
    #status=None,
    #date=datetime.date(2020, 6, 18), 
)
implementation.add_link(url=f"https://openkim.org/id/{kimid}", linktext = kimid)

index = len(potential.implementations)
potential.implementations.append(implementation)



In [15]:
potential.build_model()
potential.html(render=True)

#### 3.2.3 Select implementation

Must be done

In [8]:
index = 0
implementation = potential.implementations[index]

### 3.3 Modify implementation

#### 3.3.1 Modify implementation metadata 

In [24]:
implementation.type = 'LAMMPS pair_style hdnnp'

In [22]:
implementation.type = 'deepmd-kit'
#implementation.key=None
#implementation.date=datetime.date(2020, 1, 10)
#implementation.id = f'{potential.impid_prefix}--LAMMPS--ipr1'
#implementation.notes = ' '.join(['This file was provided by Giovanni Bonny on April 3, 2024 with the agreement of all the authors.'])
#implementation.notes += " Update July 19 2023: This version has been superseded as it had the incorrect atomic mass for H."

#implementation.status='superseded'

#### 3.3.2 Add implementation content

The content for the implementation can exist in one of the following formats

1. Artifacts: files that are hosted on the potentials repository.
2. Parameters: list of the potential's parameters to show as html text on the repository.
3. WebLinks: hyperlinks to content hosted externally.

#### Option #1: Add artifacts

File path parameters

- __newpotpath__ is the path to the directory where the new potential files are located.
- __localpath__ is the path to the local copy of the NIST potentials repository.
- __webpath__ is the url path for the NIST potentials repository that coincides with the localpath on the machine.
- __relpath__ is the relative path from localpath where the files are copied to, and the relative path from webpath where they will be found once uploaded to the website.

In [17]:
socket.gethostname()

'PG911113'

In [25]:
# Paths to files on desktop
machine = socket.gethostname()
if machine == 'PG903193':
    newpotpath = Path('F:/website/new potentials')
    localpath = Path('F:/website/IPR-website/potentials/')

# Paths to files on laptop
elif machine == 'PG911113':
    newpotpath = Path('/mnt/c/Users/lmh1/OneDrive - NIST/Documents/new_potentials')
    localpath = Path('/home/lmh1/website/IPR-website/potentials/')

# Web and relative paths
webpath = 'https://www.ctcms.nist.gov/potentials/'
relpath = f'Download/{potential.id}/{index+1}/'

Each artifact has:

- __filename__ (*str*) The name of the file (no path information).
- __url__ (*str*) The absolute url where the file can be downloaded from.
- __label__ (*str, optional*) An optional descriptive label for the file.

In [20]:
for potfile in newpotpath.glob('*'):
    if potfile.is_file():
        print(potfile.name)

lmp.in
NNIP-FeH.zip


In [26]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'NNIP-FeH.zip'),
    ]

labels = [None for i in range(len(filenames))]

In [146]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'READ_ME.txt'),
    Path(newpotpath, 'FeCCr_d.eam.alloy'),
    Path(newpotpath, 'FeCCr_s.eam.fs'),
]

labels = [
    'Documentation',
    'd_band',
    's_band'
]

In [98]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'F_Fe.spt'),
    Path(newpotpath, 'F_Cu.spt'),
    Path(newpotpath, 'F_Ni.spt'),
    Path(newpotpath, 'F_Mn.spt'),
    Path(newpotpath, 'rhoFe.spt'),
    Path(newpotpath, 'rhoCu.spt'),
    Path(newpotpath, 'rhoNi.spt'),
    Path(newpotpath, 'rhoMn.spt'),
    Path(newpotpath, 'pFeFe.spt'),
    Path(newpotpath, 'pCuCu.spt'),
    Path(newpotpath, 'pNiNi.spt'),
    Path(newpotpath, 'pMnMn.spt'),
    Path(newpotpath, 'pFeCu.spt'),
    Path(newpotpath, 'pFeNi.spt'),
    Path(newpotpath, 'pFeMn.spt'),
    Path(newpotpath, 'pCuNi.spt'),
    Path(newpotpath, 'pCuMn.spt'),
    Path(newpotpath, 'pNiMn.spt'),
    ]

labels = [
    'Fe F(ρ)',
    'Cu F(ρ)',
    'Ni F(ρ)',
    'Mn F(ρ)',
    'Fe ρ(r)',
    'Cu ρ(r)',
    'Ni ρ(r)',
    'Mn ρ(r)',
    'Fe φ(r)',
    'Cu φ(r)',
    'Ni φ(r)',
    'Mn φ(r)',
    'Fe-Cu φ(r)',
    'Fe-Ni φ(r)',
    'Fe-Mn φ(r)',
    'Cu-Ni φ(r)',
    'Cu-Mn φ(r)',
    'Ni-Mn φ(r)'
]

In [22]:
# Make download directory in local copy of website
downloadpath = Path(localpath, relpath)
if not downloadpath.is_dir():
    downloadpath.mkdir(parents=True)

# Copy files and add artifact listings
for filename, label in zip(filenames, labels):
    # Build url
    url = webpath + relpath + filename.name
    # Copy files
    shutil.copy(filename, downloadpath)
    print(Path(downloadpath, filename.name))
    
    # Add artifact listing to implementation
    implementation.add_artifact(filename=filename.name, url=url, label=label)

/home/lmh1/website/IPR-website/potentials/Download/2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H/1/NNIP-FeH.zip


#### Option #1b: Update file paths

In [43]:
#newdir= Path(localpath, relpath)
#if not newdir.is_dir():
#    newdir.mkdir(parents=True)

#for artifact in implementation.artifacts:
    # Copy artifact file to new location
#    oldpath = Path(localpath, artifact.url.replace(webpath, ''))
    
#    newpath = Path(newdir, artifact.filename)
    #shutil.copy(oldpath, newpath)
    
    # Update the url
#    artifact.url = webpath + relpath + artifact.filename

#### Option #2: Add parameters

Each parameter has:

- __name__ (*str, optional*) The name of the parameter or string parameter line.
- __value__ (*float, optional*) The value of the parameter.
- __unit__ (*str, optional*) Units associated with value.

Note: if the parameters are purely text-based, you can use only the name field.

#### Option #3: Add weblinks

Each weblink has:

- __url__ (*str*) URL for the link.
- __label__ (*str, optional*) A short description label to proceed the link.
- __linktext__ (*str, optional*) The text for the link, i.e. what gets clicked on.

In [25]:
url = linktext = 'https://docs.lammps.org/Packages_details.html#pkg-ml-hdnnp'
label = 'ML-HDNNP documentation'

implementation.add_link(url=url, label=label, linktext=linktext)



In [ ]:
#implementation.links.pop()

In [82]:
#implementation.links[0].linktext=linktext

## 4. Review and save potentials

In [26]:
potential.build_model()
potential.html(render=True)

In [27]:
potential.name

'potential.2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H'

In [10]:
db.save_potential(potential, overwrite=True, verbose=True)
with db.remote_database.cdcs.auto_set_pid_off():
    db.upload_potential(potential, workspace=workspace, overwrite=True, verbose=True)
complete_potentials.append(potential)

Potential record named potential.2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H updated in /home/lmh1/library
Potential record named potential.2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H added to https://potentials.nist.gov
Potential record named potential.2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H assigned to workspace Global Public Workspace


In [29]:
[pot.id for pot in complete_potentials]

['2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H']

In [ ]:
#db.delete_potential(potential, local=True, remote=True, verbose=True)
#complete_potentials.pop(-1)

In [ ]:
#db.download_potentials(overwrite=True, format='json', indent=4, verbose=True)

In [23]:
#complete_potentials.append(potential)

## 5. potential_LAMMPS

This section generates the potential_LAMMPS metadata records for generating the LAMMPS input command lines for supported potentials.

### 5.1. Auto-built settings

These are all settings that are universal and typically can be auto-built from the loaded potential and implementation without modification.

- __id__ (*str, optional*) implementation id. Should match with the implementation id from above.
- __key__ (*str, optional*) implementation key. Should match with the implementation key from above.
- __url__ (*str, optional*) URL to the CDCS page for the potential_LAMMPS record being built. Should end with the implementation id.
- __potid__ (*str, optional*) potential id. Should match with the potential id from above.
- __potkey__ (*str, optional*) potential key. Should match with the potential key from above.
- __poturl__ (*str, optional*) URL to the CDCS page for the Potential record generated above.
- __pair_style__ (*str*) The LAMMPS pair_style associated with the implementation.  This is extracted from the implementation.
- __dois__ (*list, optional*) A list of the DOIs from the potential's citations, auto-extracted.
- __artifacts__ (*list, optional*) A list of the artifact objects from the implementation, auto-extracted.
- __comments__ (*str, optional*) Info about where the potential is hosted, to be printed when LAMMPS commands are generated.

In [13]:
kwargs = {}
kwargs['id'] = implementation.id
kwargs['key'] = implementation.key
kwargs['url'] = f'https://potentials.nist.gov/pid/rest/local/potentials/{implementation.id}'

kwargs['potid'] = potential.id
kwargs['potkey'] = potential.key
kwargs['poturl'] = potential.url

kwargs['pair_style'] = implementation.type.replace('LAMMPS pair_style ', '')

# Auto-build list of DOIs from citations
dois = []
for citation in potential.citations:
    if citation.doi is not None:
        dois.append(citation.doi)
if len(dois) > 0:
    kwargs['dois'] = dois

if implementation.artifacts is not None:
    kwargs['artifacts'] = implementation.artifacts

kwargs['comments'] = '\n'.join([f'Potential {implementation.id} listed in the NIST Interatomic Potentials Repository:',
                                f'https://www.ctcms.nist.gov/potentials/entry/{potential.id}/{implementation.id}.html',
                                f''])

### 5.2. Specific settings

These are settings that often need manual modifications based on the potential.

- __symbols__ (*list, optional*) The particle model symbols defined by the implementation.  Optional if these match the element tags.
- __elements__ (*list, optional*) The element tags associated with each model symbol, if the symbols correspond to an element.
- __masses__ (*list, optional*) The masses to assign to each symbol.  Will default to the standard element values if elements is given and masses is not. Should be given if the implementation specifies the masses.
- __charges__ (*list, optional*) The charges to assign to each symbol. This is only for potentials where the particle models have a constant charge.
- __units__ (*str, optional*) The LAMMPS units to use with the implementation.  Default value is 'metal'.
- __atom_style__ (*str, optional*) The LAMMPS atom_style to use with the implementation.  Default value is 'atomic'.


In [14]:
kwargs['elements'] = potential.elements
#kwargs['symbols'] = ['Fe', 'nmFe']
#kwargs['masses'] = [137.34, 106.4]
#kwargs['charges'] = [0.0, 0.0]

#kwargs['units'] = 'metal'
#kwargs['atom_style'] = 'atomic'

### 5.3. Initialize the potential object

Initialization methods:

- Calling the normal init will set all of the metadata specified in the above kwargs but won't set any pair_coeff information. The pair_coeff terms can be manually built up afterwards.
- paramfile() will auto-build the pair_coeff terms for most pair_styles that rely on a single parameter file, such as eam/alloy.
- meam() will auto-build the pair_coeff terms for the meam pair_style, which relies on a libfile and optionally a paramfile.
- eim() will auto-build the pair_coeff terms for the eim pair_style, which relies on a single parameter file (but differs from the paramfile format).
- eam() will auto-build the pair_coeff terms for the eam pair_style, which relies on a parameter file for each element model.

In [28]:
# List file names for convenience
for file in implementation.artifacts:
    print(file.filename)

NNIP-FeH.zip


In [15]:
# Build potential from kwargs (and files)

#lammps_potential = PotentialLAMMPS.paramfile(paramfile='', **kwargs)
#lammps_potential = PotentialLAMMPS.meam(libfile='', paramfile='', **kwargs)
#lammps_potential = PotentialLAMMPS.eam(paramfiles=[''], **kwargs)
#lammps_potential = PotentialLAMMPS.eim(paramfile='', **kwargs)
lammps_potential = PotentialLAMMPS(**kwargs)

In [19]:
lammps_potential.build_model()

DataModelDict([('potential-LAMMPS',
                DataModelDict([('key', '0c9f600d-7bf1-4771-869a-ee28aa18c0f7'),
                               ('id', '2021--Meng-F-S--Fe-H--LAMMPS--ipr1'),
                               ('URL',
                                'https://potentials.nist.gov/pid/rest/local/potentials/2021--Meng-F-S--Fe-H--LAMMPS--ipr1'),
                               ('potential',
                                DataModelDict([('key',
                                                '738cb4ea-334a-40dd-b732-4c66ac59f060'),
                                               ('id',
                                                '2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H'),
                                               ('URL',
                                                'https://potentials.nist.gov/pid/rest/local/potentials/potential.2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H'),
                                               ('doi',
                                

### 5.4. Manually define terms for LAMMPS commands

The record is meant to capture and generate any LAMMPS commands necessary to properly use the potential.

- pair_style_terms are any terms that appear on the pair_style line after the pair_style itself.
- commands defines any additional LAMMPS command lines.  Each new line can be added with add_command().
- pair_coeffs define the pair_coeff lines.  Each new line can be added with add_pair_coeff(), and consists of an interaction list and a terms list. The list of terms is anything in the pair_coeff line after the two interaction species/wildcard options. The interaction list specifies which symbol models are connected to the pair_coeff definition. Typical variations are
    - no interaction defined to work with all symbols (will automatically add '* *')
    - two symbols for pair potentials to define the particular species-species interaction, and
    - an any-length list of symbols specifying the subset of models tied to a specific parameter file when defining a hybrid potential. 

The "terms" used for all three are specified with a term type and value.  The supported term types are:

- "option" is the basic term type that is saved as a str.
- "parameter" is for numerical terms.
- "file" is for a term that points to a parameter file for the potential.
- "symbols" indicates where the list of symbols models that correspond to the integer atom types is to be given.



In [18]:
# Define additional pair_style terms
lammps_potential.pair_style_terms.add_term('parameter', 6.5)
lammps_potential.pair_style_terms.add_term('option', 'dir')
lammps_potential.pair_style_terms.add_term('file', '.')
lammps_potential.pair_style_terms.add_term('option', 'showew')
lammps_potential.pair_style_terms.add_term('option', 'no')
lammps_potential.pair_style_terms.add_term('option', 'showewsum')
lammps_potential.pair_style_terms.add_term('parameter', 1000)
lammps_potential.pair_style_terms.add_term('option', 'resetew')
lammps_potential.pair_style_terms.add_term('option', 'yes')
lammps_potential.pair_style_terms.add_term('option', 'maxew')
lammps_potential.pair_style_terms.add_term('parameter', 1000000)
lammps_potential.pair_style_terms.add_term('option', 'cflength')
lammps_potential.pair_style_terms.add_term('parameter', 1.8897261328)
lammps_potential.pair_style_terms.add_term('option', 'cfenergy')
lammps_potential.pair_style_terms.add_term('parameter', 0.0367493254)



In [ ]:
# Manually define pair_coeff lines (if potential is not consistent with one of the class methods above).
lammps_potential.add_pair_coeff()
lammps_potential.pair_coeffs[0].add_term('symbols', True)

In [ ]:
# Define any additional required command lines
#lammps_potential.add_command()
#lammps_potential.commands[0].add_term('option', 'asfd')

### 5.2 Load potential_LAMMPS from database 

In [ ]:
#lammps_potential = db.get_lammps_potential(id='2023--Jana-R--Fe--LAMMPS--ipr1', verbose=True)

### 5.3 Load potential_LAMMPS from local file not in database 

In [44]:
#fname = Path('2020--Groger-R--Co-Cr-Fe-Mn-Ni--LAMMPS--ipr1.json')
#lammps_potential = potentials.record.PotentialLAMMPS(fname)

### 5.4 Review and save

#### Review model

In [20]:
print(lammps_potential.model.json(indent=2))

{
  "potential-LAMMPS": {
    "key": "0c9f600d-7bf1-4771-869a-ee28aa18c0f7",
    "id": "2021--Meng-F-S--Fe-H--LAMMPS--ipr1",
    "URL": "https://potentials.nist.gov/pid/rest/local/potentials/2021--Meng-F-S--Fe-H--LAMMPS--ipr1",
    "potential": {
      "key": "738cb4ea-334a-40dd-b732-4c66ac59f060",
      "id": "2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H",
      "URL": "https://potentials.nist.gov/pid/rest/local/potentials/potential.2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H",
      "doi": "10.1103/physrevmaterials.5.113606"
    },
    "comments": "Potential 2021--Meng-F-S--Fe-H--LAMMPS--ipr1 listed in the NIST Interatomic Potentials Repository:\nhttps://www.ctcms.nist.gov/potentials/entry/2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H/2021--Meng-F-S--Fe-H--LAMMPS--ipr1.html\n",
    "units": "metal",
    "atom_style": "atomic",
    "atom": [
      {
        "symbol": "Fe",
        "element": "Fe"
      },
      {
        "symbol": "H",
        "element": "H"
      }
    ],
    "pair_s

In [22]:
print(lammps_potential.pair_info())

print "Potential 2021--Meng-F-S--Fe-H--LAMMPS--ipr1 listed in the NIST Interatomic Potentials Repository:"
print "https://www.ctcms.nist.gov/potentials/entry/2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H/2021--Meng-F-S--Fe-H--LAMMPS--ipr1.html"
print "Publication(s) related to the potential:"
print "https://doi.org/10.1103/physrevmaterials.5.113606"
print "Parameter file(s) can be downloaded at:"
print "https://www.ctcms.nist.gov/potentials/Download/2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H/1/NNIP-FeH.zip"

pair_style hdnnp 6.5 dir . showew no showewsum 1000.0 resetew yes maxew 1000000.0 cflength 1.8897261328 cfenergy 0.0367493254
pair_coeff * * Fe H

mass 1 55.845
mass 2 1.007975




In [30]:
lammps_potential.name = lammps_potential.id

#### Save

In [31]:
db.save_lammps_potential(lammps_potential, filenames=filenames,  verbose=True)
with db.remote_database.cdcs.auto_set_pid_off():
    db.upload_lammps_potential(lammps_potential, workspace=workspace, overwrite=True, verbose=True)

potential_LAMMPS record named 2021--Meng-F-S--Fe-H--LAMMPS--ipr1 added to /home/lmh1/library
files saved to local folder
potential_LAMMPS record named 2021--Meng-F-S--Fe-H--LAMMPS--ipr1 added to https://potentials.nist.gov
potential_LAMMPS record named 2021--Meng-F-S--Fe-H--LAMMPS--ipr1 assigned to workspace Global Public Workspace


In [ ]:
#db.delete_lammps_potential(lammps_potential, local=True, remote=True, verbose=True)

In [ ]:
#db.download_lammps_potentials(getfiles=True, overwrite=True, verbose=True)

- - -

## 5. Actions

Actions are used by the website to note changes in content and representation. 

### 5.1 Build action

#### Option #1: new posting(s)

In [30]:
for complete_potential in complete_potentials:
    print(complete_potential.id)

2021--Meng-F-S-Du-J-P-Shinzato-S-et-al--Fe-H


In [31]:
action = potentials.load_record('Action',
    type = 'new posting',
    potentials = complete_potentials,
    #date = datetime.date(2021, 10, 26),
    comment = "New potential for Fe-H",
)

KeyError: 'Unknown Value style list_contains'

#### Option #2: updated posting(s)

In [39]:
action = potentials.load_record('Action',
    type = 'updated posting',
    potentials = complete_potentials,
    date = None,
    comment = 'New version added that fixes the atomic mass of H',
)

#### Option #3: retraction

In [ ]:
action = potentials.load_record('Action',
    type = 'retraction',
    potentials = complete_potentials,
    date = None,        
    comment =,
)

#### Option #4: site change

In [ ]:
action = potentials.load_record('Action',
    type = 'site change',
    date = None,
    comment = 'IDs added for all non-LAMMPS implementations. The handling of fictional potentials made consistent with "real" potentials.',
    potentials = []
)

### 5.2 Look and save

In [33]:
action.build_model()
action.html(render=True)

In [34]:
db.upload_action(action, workspace=workspace, overwrite=True, verbose=True)

Action record named 2024-08-27 New pace potential for hydrocarbons added to https://potentials.nist.gov/
Action record named 2024-08-27 New pace potential for hydrocarbons assigned to workspace Global Public Workspace


- - -

## 6. Requests

Requests are used by the website to list when people are looking for specific models that we don't have.

### 6.1 Build request

In [42]:
request = potentials.load_record(
    'Request',
    #comment = 'for WS2 films',
    #date = datetime.date(2022, 11, 1),
    systems = [
        {
        #    'formula': 'KNbO3',
            'elements': ['W','Co', 'C'],
        },
        #{
        #    'formula': 'RbNbO3',
        #    'elements': ['Rb', 'Nb', 'O'],
        #},
    ]
)


### 6.2 Look and save

In [43]:
request.build_model()
display(HTML(request.html()))

In [44]:
db.upload_request(request, workspace=workspace, overwrite=True, verbose=True)

Request record named 2024-03-21 W Co C added to https://potentials.nist.gov/
Request record named 2024-03-21 W Co C assigned to workspace Global Public Workspace


## 7. Related models

In [105]:
import json

def add_related_models(localpath, interaction, model1, model2):
    
    with open(Path(localpath, 'site/related-interactions.json')) as f:
        related_models = json.load(f)
    
    # If no shared models for that interaction yet
    if interaction not in related_models:
        print('new interaction')
        related_models[interaction] = [[model1, model2]]
            
    else:
        int_models = related_models[interaction]
        
        # Search for set with one of the elements
        match = False
        for i in range(len(int_models)):
            modelset = int_models[i]
            if model1 in modelset:
                if match is False:
                    match = i
                else:
                    raise ValueError('given models found in multiple sets!')
            if model2 in modelset:
                if match is False or match == i:
                    match = i
                else:
                    raise ValueError('given models found in multiple sets!')
                    
        if match is False:
            print('new set')
            int_models.append(sorted([model1, model2]))
        else:
            if model1 not in int_models[match]:
                int_models[match] = sorted(int_models[match] + [model1])
                print('model1 added to existing set')
            elif model2 not in int_models[match]:
                int_models[match] = sorted(int_models[match] + [model2])
                print('model2 added to existing set')
            else:
                print('both models already in existing set')
    
    with open(Path(localpath, 'site/related-interactions.json'), 'w') as f:
        json.dump(related_models, fp=f)

In [111]:
interaction = 'Ni'
model1 = '2022--Mendelev-M-I--Ni-Al'
model2 = '2024--Mendelev-M-I--Ni-Al-Cr'
machine = 'desktop'

if machine == 'desktop':
    localpath = Path('F:/website/IPR-website/potentials/')
elif machine == 'laptop':
    localpath = Path('C:/Users/lmh1/Documents/website/IPR-website/potentials/')

add_related_models(localpath, interaction, model1, model2)

model2 added to existing set
